**Classification of XRays using Machine Learning**
---
My name is Teddy John Kamau Mbiri, a final year Computer Science student at University of West London. 
This is my final year project, the use of Machine Learning to classify X-rays, whether a patient has pneumonia, covid pneumonia or neither.


**Data Exploration**
---
The section below will explore the data and the features of the dataset.
I will be using data from multiple sources and previous work to create a model that can classify X-rays.

**Data Source**
1.	https://drive.google.com/drive/folders/128hxYxQ8kVEkSkVGikueiqBxWTmDIW2h
2.	https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
3.	https://www.kaggle.com/nikhilpandey360/chest-xray-masks-and-labels
4.  https://data.mendeley.com/datasets/jctsfj2sfn/1
5. https://www.rsna.org/education/ai-resources-and-training/ai-image-challenge/rsna-pneumonia-detection-challenge-2018



In [1]:
#split the data into train and test and validation 
#import splitfolders
#Library required for the splitfolders function
#splitfolders.ratio("C:/Users/tedjo/Pictures/XRays", output="C:/Users/tedjo/Pictures/XRays/SplitData", seed=1337, ratio=(.8, .1, .1))
#SplitData is the folder where the data will be stored